In [ ]:
args <- commandArgs(TRUE)
rdsfile <- args[1]     # Expression data of oocytes merged with the 5-10 nearest spots to the oocyte center, expression data of granulosa cells randomly merged around oocytes at the same stage, and expression data of TC cells randomly merged around oocytes at the same stage
output_dir <- args[2]  # Output file path
prefix <- args[3] # Prefix for output file
genesetsfile <- args[4] # gene set file

# 01. standard analysis

In [ ]:
micro <- readRDS(rdsfile)

In [ ]:
micro <- NormalizeData(micro, normalization.method = "LogNormalize", scale.factor = 10000)
micro <- FindVariableFeatures(micro, selection.method = "vst", nfeatures = 10000)
micro <- ScaleData(micro,vars.to.regress =c("percent.mt","nCount_RNA"))
micro <- RunPCA(micro,features = VariableFeatures(object =micro),npcs=50, verbose=FALSE)
micro <- RunUMAP(micro,dims = 1:50)
micro <- FindNeighbors(micro, dims = 1:50)
meta <- FetchData(object =micro, vars = c("PC_1","PC_2","UMAP_1","UMAP_2","tSNE_1","tSNE_2","orig.ident","subtype"))
meta$orig.ident=factor(meta$orig.ident, levels=c("OO","GC","TC"))
saveRDS(micro,file=paste0("OO+GC+TC.dis20.final.rds"))

# 02. marker genes identified

In [9]:
micro@meta.data$orig.ident <- factor(micro@meta.data$orig.ident, levels = c("OO", "GC", "TC"))

In [32]:
Idents(micro)=micro@meta.data$orig.ident
markers2 <- FindAllMarkers(micro, only.pos =FALSE, min.pct = 0.1)
write.csv(markers2, file =paste0(output_dir,"OO+GC+TC.dis20.markergene.xls"))

Calculating cluster OO

Calculating cluster GC

Calculating cluster TC



# 03. gene sets score

In [ ]:
geneset=read.table(genesetsfile,header=T,sep="\t")
geneset_list <- split(geneset$Gene, geneset$Module)

In [ ]:
micro.score=micro
for (module in names(geneset_list)) {
    micro.score <- AddModuleScore(micro.score,features = list(geneset_list[[module]]),name=module,n.bin = 500,assay="RNA",ctrl.size=500)
}
keys <- names(geneset_list)
modified_keys <- paste(keys, "1", sep="")


In [9]:
meta <- FetchData(object =micro.score, vars = c("orig.ident",modified_keys))

In [ ]:
options(repr.plot.width =3.5,repr.plot.height =3.2)
p= DotPlot(object = micro.score, features =modified_keys,scale.by = "size",assay="RNA",group.by ="orig.ident")+RotatedAxis() + labs(x="",y="") + scale_fill_manual(values = allcol)+scale_color_gradient2(low = "blue",mid="grey95",high = "red",guide = guide_colorbar(ticks.colour = "black",frame.colour = "black"),name = "Average\nexpression")+cowplot::theme_cowplot()+mytheme_dot
ggsave(paste0(output_dir,"OO+GC+TC.dis20",".score.pdf"),plot=p,w=3.5,h=3.2)
p

In [55]:
micro.score@meta.data$subtype2=paste0(micro.score@meta.data$orig.ident,"-",micro.score@meta.data$subtype)

In [56]:
micro.score@meta.data$subtype2 <- factor(micro.score@meta.data$subtype2,levels = c("OO-C1","OO-C2","OO-C3","OO-C4","GC-C1","GC-C2","GC-C3","GC-C4","TC-C1","TC-C2","TC-C3","TC-C4"))

In [ ]:
options(repr.plot.width =4.5,repr.plot.height =4.5)
p= DotPlot(object = micro.score, features =modified_keys,scale.by = "size",assay="RNA",group.by ="subtype2")+RotatedAxis() + labs(x="",y="") + scale_fill_manual(values = allcol)+scale_color_gradient2(low = "blue",mid="grey95",high = "red",guide = guide_colorbar(ticks.colour = "black",frame.colour = "black"),name = "Average\nexpression")+cowplot::theme_cowplot()+mytheme_dot
ggsave(paste0(output_dir,"OO+GC+TC.dis20",".score2.pdf"),plot=p,w=4.5,h=4.5)
p